# Predict Calorie Expenditure

### *Playground Series - Season 5, Episode 5*  
  
##### **Dataset Description**

The dataset for this competition (both train and test) was generated from a deep learning model trained on the Calories Burnt Prediction dataset. Feature distributions are close to, but not exactly the same, as the original. Feel free to use the original dataset as part of this competition, both to explore differences as well as to see whether incorporating the original in training improves model performance.

###### **Files**

*    `train.csv` - the training dataset; Calories is the continuous target
*    `test.csv` - the test dataset; your objective is to predict the Calories for each row
*    `sample_submission.csv` - a sample submission file in the correct format.

##### **Evaluation** 

The evaluation metric for this competition is **Root Mean Squared Logarithmic Error**. 

The **RMSLE** is calculated as: 

$$
\text{RMSLE} = \left( \frac{1}{n} \sum_{i=1}^n \left(\log (1 + \hat{y}_i) - \log (1 + y_i)\right)^2 \right)^{\frac{1}{2}}
$$

where: 

* $n$ is the total number of observations in the test set,
* $\hat{y}_i$ is the predicted value of the target for instance (i),
* $y_i$ is the actual value of the target for instance (i), and, 
* $\log$ is the natural logarithm.


### Exploratory Data Analysis

We see from the Kaggle contest page the distribution of values within fields. 

In [66]:
import pandas as pd 

url = 'https://raw.githubusercontent.com/maggieclark/kaggle-calories/refs/heads/main/train.csv'
df = pd.read_csv(url)

df.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          750000 non-null  int64  
 1   Sex         750000 non-null  object 
 2   Age         750000 non-null  int64  
 3   Height      750000 non-null  float64
 4   Weight      750000 non-null  float64
 5   Duration    750000 non-null  float64
 6   Heart_Rate  750000 non-null  float64
 7   Body_Temp   750000 non-null  float64
 8   Calories    750000 non-null  float64
dtypes: float64(6), int64(2), object(1)
memory usage: 51.5+ MB


In [68]:
df.describe()

,id,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
count,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000
mean,374999.500000,41.420404,174.697685,75.145668,15.421015,95.483995,40.036253,88.282781
std,216506.495284,15.175049,12.824496,13.982704,8.354095,9.449845,0.779875,62.395349
min,0.000000,20.000000,126.000000,36.000000,1.000000,67.000000,37.100000,1.000000
25%,187499.750000,28.000000,164.000000,63.000000,8.000000,88.000000,39.600000,34.000000
50%,374999.500000,40.000000,174.000000,74.000000,15.000000,95.000000,40.300000,77.000000
75%,562499.250000,52.000000,185.000000,87.000000,23.000000,103.000000,40.700000,136.000000
max,749999.000000,79.000000,222.000000,132.000000,30.000000,128.000000,41.500000,314.000000


#### **Pre-processing Steps:** 

* drop `id` column
* drop duplicate rows
* encode categorical variable
* normalize numeric variables
* 80/20 split of training/testing

In [69]:
# drop id column and duplicates
df = df.drop_duplicates().drop('id',axis=1)
df.head()

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


In [70]:
# encode categorical features, since Sex is male femail we can encode it numerically
df['Sex'] = df['Sex'].map({'male':0, 'female':1})

# normalize numeric features 
from sklearn.preprocessing import StandardScaler

features = ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

In [71]:
df.head()

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,-0.357192,1.115235,0.490201,1.266324,0.583714,1.235772,150.0
1,1,1.487943,-0.912137,-1.083172,-0.888309,-1.109436,-0.431163,34.0
2,1,0.631273,-1.068088,-0.797104,-1.008011,-1.215258,-0.302938,29.0
3,0,-1.411555,1.349162,1.062337,1.146622,1.007002,0.851095,140.0
4,1,-0.225397,-0.678209,-1.011655,1.146622,0.689536,0.722869,146.0


In [72]:
# split into training and testing sets 
from sklearn.model_selection import train_test_split 

X = df.drop('Calories', axis=1)
y = df['Calories']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

##### **Linear Regression**

Linear regression raised an error when calculating RMSLE because some of the predicted values were negative or zero. To handle this, we transform the target variable using a log(1 + y) tranformation to prevent zero values before training, train and predict in log space, then back-transform the predictions to the origianl scale before calculating RMSLE. 

In [73]:
# linear regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error, r2_score
import numpy as np

# model name 
lr_modelName = 'Linear Regression'

# RMSLE scorer (sklearn doesn't include RMSLE directly)
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

# Apply log(1 + y) transformation to the target variables
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

# Train the model on the transformed target
model = LinearRegression()
model.fit(X_train, y_train_log)

# Predict in log space
y_pred_log = model.predict(X_test)

# Back-transform predictions to original scale
y_pred = np.expm1(y_pred_log)  # exp(y_pred_log) - 1

lr_rmsle = rmsle(y_test, y_pred)
lr_r2 = r2_score(y_test, y_pred)

print(f"RMSLE: {lr_rmsle}")
print(f"R-squared: {lr_r2}")

RMSLE: 0.1800491400832319
R-squared: 0.9163635765400516


The **Root Mean Squared Logarithmic Error (RMSLE)** measures the average logarithmic difference between predicted and actual values. 

The **$R^2$** value measures amount of variance in the target variable explained by the model, in this case, about 8.5% of the variability is unexplained. 

##### **Random Forest**

In [74]:
# random forest 
from sklearn.ensemble import RandomForestRegressor

# model name 
rf_modelName = 'Random Forest' 

# Log-transform target variable
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

# Train model on log-transformed target
model = RandomForestRegressor(random_state=99, n_jobs=-1)
model.fit(X_train, y_train_log)

# Predict in log space
y_pred_log = model.predict(X_test)

# Back-transform predictions to original scale
y_pred = np.expm1(y_pred_log)

# Evaluate the model
rf_rmsle = rmsle(y_test, y_pred)
rf_r2 = r2_score(y_test, y_pred)

print(f"RMSLE: {rf_rmsle}")
print(f"R-squared: {rf_r2}")

RMSLE: 0.06426749069771626
R-squared: 0.9961135686185102


#### **Setup Cross-Validation**

Cross-validation is used to evaluate the performance and generalizability of the model. 

* Rather than testing the model on a single train-test split, K-fold cross-validation tests the model on multiple train-test splits, to give a better estimate of how the model performs on unseen data.
* By validating the model on different subsets, the risk of the model overfitting to the data is reduced.

In [75]:
# # set up cross-validation
# from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
# from sklearn.metrics import make_scorer, mean_squared_log_error
# from sklearn.model_selection import KFold

# # Model name
# rfCV_modelName = 'Random Forest with Cross-Validation'

# # Prepare KFold cross-validator
# kf = KFold(n_splits=5, shuffle=True, random_state=99)

# rmsle_scores = []
# r2_scores = []

# for train_index, test_index in kf.split(X):
#     X_train_cv, X_test_cv = X.iloc[train_index], X.iloc[test_index]
#     y_train_cv, y_test_cv = y.iloc[train_index], y.iloc[test_index]


#     # Log-transform target
#     y_train_log = np.log1p(y_train_cv)

#     # Train model
#     model = RandomForestRegressor(random_state=99, n_jobs=-1)
#     model.fit(X_train_cv, y_train_log)

#     # Predict in log space
#     y_pred_log = model.predict(X_test_cv)

#     # Back-transform predictions
#     y_pred = np.expm1(y_pred_log)

#     # Evaluate
#     rmsle_scores.append(rmsle(y_test_cv, y_pred))
#     r2_scores.append(r2_score(y_test_cv, y_pred))

# # Compute average metrics
# rfCV_rmsle = np.mean(rmsle_scores)
# rfCV_r2 = np.mean(r2_scores)

# print(f"Average RMSLE: {rfCV_rmsle}")
# print(f"Average R-squared: {rfCV_r2}")

##### Extreme Gradient Boosting 

In [76]:
import xgboost as xgb

# Log-transform target variable
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

# Initialize XGBoost regressor
model = xgb.XGBRegressor(random_state=99, n_jobs=-1)

# Train on log-transformed target
model.fit(X_train, y_train_log)

# Predict in log space
y_pred_log = model.predict(X_test)

# Back-transform predictions to original scale
y_pred = np.expm1(y_pred_log)

# Calculate RMSLE and R²
xgb_rmsle = rmsle(y_test, y_pred)
xgb_r2 = r2_score(y_test, y_pred)

print(f"RMSLE: {xgb_rmsle}")
print(f"R-squared: {xgb_r2}")

RMSLE: 0.06259504517589733
R-squared: 0.9961400344777009


##### Using grid search to find best hyperparameters:

In [77]:
from xgboost import XGBRegressor, DMatrix
import random
import itertools

# Log-transform target variable
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

# Define parameter space
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.01, 0.1, 1],
    'reg_lambda': [0.1, 1, 10, 20]
}

# Choose a fixed number of random parameter sets to evaluate
n_samples = 25
random_params = [dict(zip(param_grid.keys(), values)) 
                 for values in random.sample(list(itertools.product(*param_grid.values())), n_samples)]

# Create a holdout validation set for early stopping
X_train_split, X_valid_split, y_train_split, y_valid_split = train_test_split(X_train, y_train_log, test_size=0.2, random_state=99)

# Prepare to track the best model
best_model = None
best_score = float('inf')
best_params = None

# Iterate over the random parameter sets
for param_set in random_params:
    # Initialize model with parameters
    model = XGBRegressor(objective='reg:squarederror', random_state=99, n_jobs=-1, **param_set)
    
    # Train the model with early stopping
    model.fit(X_train_split, y_train_split, 
              verbose=False)
    
    # Predict in log space
    y_pred_log = model.predict(X_test)
    y_pred = np.expm1(y_pred_log)
    
    # Calculate RMSLE
    rmsle_score = np.sqrt(mean_squared_log_error(y_test, y_pred))
    
    # Update the best model if this one is better
    if rmsle_score < best_score:
        best_model = model
        best_score = rmsle_score
        best_params = param_set

# Final evaluation
xgb2_r2 = r2_score(y_test, np.expm1(best_model.predict(X_test)))

print(f"Best Parameters: {best_params}")
print(f"RMSLE: {best_score}")
print(f"R-squared: {xgb2_r2}")


Best Parameters: {'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.05, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 1, 'reg_lambda': 20}
RMSLE: 0.060593089141339984
R-squared: 0.9965802059318157


In [78]:
# Re-train best model on full training data
final_model = XGBRegressor(objective='reg:squarederror', random_state=99, n_jobs=-1, **best_params)
final_model.fit(X_train, y_train_log, verbose=False)

# Predict on test set (log space)
y_pred_log_final = final_model.predict(X_test)
y_pred_final = np.expm1(y_pred_log_final)  # back-transform

# Calculate final metrics
xgb3_rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred_final))
xgb3_r2 = r2_score(y_test, y_pred_final)

print("Re-trained final model metrics:")
print(f"RMSLE: {xgb3_rmsle}")
print(f"R-squared: {xgb3_r2}")


Re-trained final model metrics:
RMSLE: 0.06053329796054797
R-squared: 0.996613944371695
